The code from `march1.ipynb` is modified in one overarching respect: The input csv-file contains both descrete and continous features (X columns). The indeces of the descrete columns (as well as name of the file) are hardcoded below but they are of course meant to become user-provided arguments. The output is a dictionary, where the keys are products of dimensions of contingency_matrices encountered in the computation, and values are again dictionaries similar to the one used before as a whole output i.e. maps from column-indeces to maximal information gains (and corresponding tuples).

In order to produce an input file, every `n`th column of `madelon.csv` will be discretized. Details of the way the partially-discretized csv is generated is fairly obvious form the code below. The resulting file is saved as a new file `madelon_n.csv` and the orginal `madelon.csv` is not used anymore.

---
Reminder:
* Swap error handling in `matrix.h` for assertions.

---
The `cpp` code

In [3]:
%%writefile wrap_discretize.cpp
/*
<%
cfg['compiler_args'] = ['-std=c++11', '-fopenmp']
cfg['linker_args'] = ['-fopenmp']
setup_pybind11(cfg)
%>
*/

#include <algorithm>
#include <cstdint>
#include <cstring>
#include <random>
#include <vector>
#include <pybind11/pybind11.h>
//#include <pybind11/stl.h>
#include <pybind11/numpy.h>

namespace py = pybind11;
using namespace pybind11::literals;

void discretize(
    uint32_t seed,
    uint32_t discretization_index,
    uint32_t feature_id,
    std::size_t divisions,
    std::size_t object_count,
    py::array_t<double> py_in_data,
    py::array_t<double> py_sorted_in_data,
    py::array_t<uint8_t> py_out_data,
    double range_
) {
    
    // the python part
    py::buffer_info py_in_data_buf = py_in_data.request();
    auto *in_data = static_cast<const double *>(py_in_data_buf.ptr);    
    
    py::buffer_info py_sorted_in_data_buf = py_sorted_in_data.request();
    auto *sorted_in_data = static_cast<const double *>(py_sorted_in_data_buf.ptr);
    
    py::buffer_info py_out_data_buf = py_out_data.request();
    auto *out_data = static_cast<uint8_t *>(py_out_data_buf.ptr);
    
    // end of the python part
    
    double* thresholds = new double[divisions];

    // brackets to limit scope
    {
        double sum = 0.0f;
        // brackets to limit scope
        {
            std::mt19937 seed_random_generator0(seed);
            std::mt19937 seed_random_generator1(seed_random_generator0() ^ discretization_index);
            std::mt19937 random_generator(seed_random_generator1() ^ feature_id);

            // E(X) = (a + b) / 2 = (1 - range + 1 + range) / 2 = 1
            std::uniform_real_distribution<double> uniform_range(1.0f - range_, 1.0f + range_);

            for (std::size_t d = 0; d < divisions; ++d) {
                thresholds[d] = uniform_range(random_generator);
                sum += thresholds[d];
            }

            sum += uniform_range(random_generator);
        }

        std::size_t done = 0;
        const double length_step = static_cast<double>(object_count) / sum;

        // thresholds are converted from an arbitrary space into real values (via indices)
        // d - iterates over divisions (of a variable in a discretization)
        for (std::size_t d = 0; d < divisions; ++d) {
            done += std::lround(thresholds[d] * length_step);

            // Note: Check when will this happen, maybe could be skipped
            if (done >= object_count) {
                done = object_count - 1;
            }

            thresholds[d] = sorted_in_data[done];
        }
    }

    // o - iterates over objects
    for (std::size_t o = 0; o < object_count; ++o) {
        out_data[o] = 0;

        // out_data[o] (starting with 0) is incremented every time in_data[o] is above a threshold
        // divisions is a small number (<=15), no reason to use binsearch, hence linear
        // d - iterates over divisions (per object o)
        for (std::size_t d = 0; d < divisions; ++d) {
            out_data[o] += in_data[o] > thresholds[d];
        }
    }

    delete[] thresholds;
}


PYBIND11_MODULE(wrap_discretize, module) {
    module.doc() = "Wrapper of https://bitbucket.org/mdfs/mdfs/src/master/src/cpu/discretize.cpp";
    module.def("discretize", &discretize,
               "In place discretization.",
               "seed"_a,
               "discretization_index"_a,
               "feature_id"_a,
               "divisions"_a,
               "object_count"_a,
               "py_in_data"_a,
               "py_sorted_in_data"_a,
               "py_out_data"_a,
               "range_"_a
              );
}

Overwriting wrap_discretize.cpp


In [1]:
%%writefile fast.cpp
/*
<%
cfg['compiler_args'] = ['-std=c++11', '-fopenmp']
cfg['linker_args'] = ['-fopenmp']
setup_pybind11(cfg)
%>
*/

#include <math.h>
#include <tuple>
//#include <vector>
#include <pybind11/pybind11.h>
//#include <pybind11/stl.h>
#include <pybind11/numpy.h>
#include <omp.h>

#include "matrix_class.h"

namespace py = pybind11;


std::tuple<double, double, double> work_3a(const int kData_dim,
                                           const std::tuple<int, int, int> py_Xdims,
                                           py::array_t<int> &py_X0,
                                           py::array_t<int> &py_X1,
                                           py::array_t<int> &py_X2,
                                           const int kN_classes,
                                           py::array_t<double> &py_pseudo_counts,
                                           py::array_t<int> &py_y) {
    
    const int kC_Xdim_0 = std::get<0>(py_Xdims);
    const int kC_Xdim_1 = std::get<1>(py_Xdims);
    const int kC_Xdim_2 = std::get<2>(py_Xdims);
    const int kC_ydim = kN_classes;
    
    py::buffer_info py_X0_buf = py_X0.request();
    auto *X0 = static_cast<int *>(py_X0_buf.ptr);
    py::buffer_info py_X1_buf = py_X1.request();
    auto *X1 = static_cast<int *>(py_X1_buf.ptr);
    py::buffer_info py_X2_buf = py_X2.request();
    auto *X2 = static_cast<int *>(py_X2_buf.ptr);
    
    py::buffer_info py_pseudo_counts_buf = py_pseudo_counts.request();
    auto *pseudo_counts = (double *) py_pseudo_counts_buf.ptr;
    
    py::buffer_info py_y_buf = py_y.request();
    auto *y = (int *) py_y_buf.ptr;
    
    Matrix<int> contingency_m(kC_Xdim_0, kC_Xdim_1, kC_Xdim_2, kC_ydim);
    for (int data_idx = 0; data_idx < kData_dim; data_idx++) {
        ++contingency_m(X0[data_idx], X1[data_idx], X2[data_idx], y[data_idx]);
    }
    
    double neg_H = 0., neg_H_X0 = 0., neg_H_X1 = 0., neg_H_X2 = 0.;
    
    #pragma omp parallel
    {
    #pragma omp sections
    {
        #pragma omp section
        {
        double local_neg_H = 0.;
        for (int C_Xidx_0 = 0; C_Xidx_0 < kC_Xdim_0; ++C_Xidx_0) {
            for (int C_Xidx_1 = 0; C_Xidx_1 < kC_Xdim_1; ++C_Xidx_1) {
                for (int C_Xidx_2 = 0; C_Xidx_2 < kC_Xdim_2; ++C_Xidx_2) {
                    double count_y = 0.;
                    for (int C_yidx = 0; C_yidx < kC_ydim; ++C_yidx) {
                        double count = pseudo_counts[C_yidx] + contingency_m(C_Xidx_0, C_Xidx_1, C_Xidx_2, C_yidx);
                        local_neg_H += count * log2(count);
                        count_y += count;
                    }
                    local_neg_H -= count_y * log2(count_y);
                }
            }
        }
        #pragma omp atomic
            neg_H += local_neg_H;
        }        
        
        #pragma omp section
        {
        double local_neg_H_X0 = 0.;
        for (int C_Xidx_1 = 0; C_Xidx_1 < kC_Xdim_1; C_Xidx_1++) {
            for (int C_Xidx_2 = 0; C_Xidx_2 < kC_Xdim_2; C_Xidx_2++) {
                double count_y_X0 = 0.;
                for (int C_yidx = 0; C_yidx < kC_ydim; C_yidx++) {
                    double count_X0 = 0.;
                    for (int C_Xidx_0 = 0; C_Xidx_0 < kC_Xdim_0; C_Xidx_0++) {
                        double count = pseudo_counts[C_yidx] + contingency_m(C_Xidx_0, C_Xidx_1, C_Xidx_2, C_yidx);
                        count_X0 += count;
                    }
                    local_neg_H_X0 += count_X0 * log2(count_X0);
                    count_y_X0 += count_X0;
                }
                local_neg_H_X0 -= count_y_X0 * log2(count_y_X0);
            }
        }
        #pragma omp atomic
            neg_H_X0 += local_neg_H_X0;
        }
        
        #pragma omp section
        {
        double local_neg_H_X1 = 0.;
        for (int C_Xidx_0 = 0; C_Xidx_0 < kC_Xdim_0; C_Xidx_0++) {
            for (int C_Xidx_2 = 0; C_Xidx_2 < kC_Xdim_2; C_Xidx_2++) {
                double count_y_X1 = 0.;
                for (int C_yidx = 0; C_yidx < kC_ydim; C_yidx++) {
                    double count_X1 = 0.;
                    for (int C_Xidx_1 = 0; C_Xidx_1 < kC_Xdim_1; C_Xidx_1++) {
                        double count = pseudo_counts[C_yidx] + contingency_m(C_Xidx_0, C_Xidx_1, C_Xidx_2, C_yidx);
                        count_X1 += count;
                    }
                    local_neg_H_X1 += count_X1 * log2(count_X1);
                    count_y_X1 += count_X1;
                }
                local_neg_H_X1 -= count_y_X1 * log2(count_y_X1);
            }
        }
        #pragma omp atomic
            neg_H_X1 += local_neg_H_X1;
        }
        
        #pragma omp section
        {
        double local_neg_H_X2 = 0.;
        for (int C_Xidx_0 = 0; C_Xidx_0 < kC_Xdim_0; C_Xidx_0++) {
            for (int C_Xidx_1 = 0; C_Xidx_1 < kC_Xdim_1; C_Xidx_1++) {
                double count_y_X2 = 0.;
                for (int C_yidx = 0; C_yidx < kC_ydim; C_yidx++) {
                    double count_X2 = 0.;
                    for (int C_Xidx_2 = 0; C_Xidx_2 < kC_Xdim_2; C_Xidx_2++) {
                        double count = pseudo_counts[C_yidx] + contingency_m(C_Xidx_0, C_Xidx_1, C_Xidx_2, C_yidx);
                        count_X2 += count;
                    }
                    local_neg_H_X2 += count_X2 * log2(count_X2);
                    count_y_X2 += count_X2;
                }
                local_neg_H_X2 -= count_y_X2 * log2(count_y_X2);
            }
        }
        #pragma omp atomic
            neg_H_X2 += local_neg_H_X2;
        }        
    }
    }
    
    return std::make_tuple(neg_H - neg_H_X0,
                           neg_H - neg_H_X1,
                           neg_H - neg_H_X2
                          ); 
}
       

PYBIND11_MODULE(fast, module) {
    module.def("work_3a", &work_3a);
}

Overwriting fast.cpp


Compilation

In [20]:
import cppimport
#cppimport.set_quiet(False)

fast = cppimport.imp("fast")
cppimport.imp("wrap_discretize")

<module 'wrap_discretize' from '/home/olszewskip/Desktop/git-repos/MDFS_playground/python/scheduler/ver0/wrap_discretize.cpython-35m-x86_64-linux-gnu.so'>

---
Make the partially-discretized csv file. Discretize every `n`th column.

In [6]:
import wrap_discretize
import numpy as np
from pandas import read_csv
import pandas as pd

data_df = read_csv("madelon.csv", header=None)
data_df_2 = data_df.copy()
in_data = np.ascontiguousarray(np.array(data_df.values.T, dtype='float64')[:-2])
out_data = np.zeros_like(in_data, dtype='uint8')

In [8]:
n = 5

for col_idx in range(0, in_data.shape[0] - 2):
    if col_idx % n == 0:
        wrap_discretize.discretize(
            seed = 123,
            discretization_index = 0,
            feature_id = col_idx,
            divisions = 1 + col_idx // n % n,
            object_count = in_data.shape[1],
            py_in_data = in_data[col_idx],
            py_sorted_in_data = np.sort(in_data[col_idx]),
            py_out_data = out_data[col_idx],
            range_ = 0
        )
        data_df_2[col_idx] = out_data[col_idx]

data_df_3 = pd.DataFrame(data_df_2.values.astype('float'))
data_df_3.to_csv('madelon_{}.csv'.format(n), header=False, index=False)

---
Make a data sample: pick `m` last X-columns in the `madelon_5`

In [9]:
m = 50

from pandas import read_csv

madelon = read_csv("madelon_5.csv", header=None)
madelon.iloc[:,-(m + 2):].to_csv('madelon_5_tiny.csv', header=False, index=False)
madelon_5_tiny = read_csv("madelon_5_tiny.csv", dtype = 'float64', header=None)
madelon_5_tiny.shape

discrete_cols = list(range(0, m, 5))
discrete_cols

[0, 5, 10, 15, 20, 25, 30, 35, 40, 45]

---
Run the thing synchronously (make it a standalone script for profiling)

In [11]:
%%writefile march2_n1.py

import csv
import numpy as np
from scipy.stats import rankdata
from pandas import read_csv, DataFrame
from itertools import product, chain, starmap, combinations, combinations_with_replacement
from time import time
import pickle

import wrap_discretize
import fast

time0 = time()

k = 3
divisions = 7
range_ = 0.00
seed = 123
wrap_discretize_switch = True

# 1. Read in the data

file = "madelon_5_tiny.csv"
discrete_cols = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45]

input_ = []
with open(file) as csvfile:
    reader = csv.reader(csvfile, delimiter=',',
                        quoting=csv.QUOTE_NONNUMERIC)
    for row in reader:
        input_.append(row)
        
input_ = np.ascontiguousarray(np.array(input_, dtype='float64').T[:-1])
data = np.zeros_like(input_, dtype='uint8')
bucket_counts = np.empty(input_.shape[0] - 1, dtype='int')

# Discretize the data

if wrap_discretize_switch:

    for col_idx in range(input_.shape[0] - 1):
        if col_idx in discrete_cols:
            data[col_idx] = input_[col_idx].astype('uint8')
            bucket_counts[col_idx] = len(np.unique(data[col_idx]))
        else:
            wrap_discretize.discretize(
                seed = 123,
                discretization_index = 0,
                feature_id = col_idx,  # ?
                divisions = divisions,
                object_count = input_.shape[1],
                py_in_data = input_[col_idx],
                py_sorted_in_data = np.sort(input_[col_idx]),
                py_out_data = data[col_idx],
                range_ = range_
            )
            bucket_counts[col_idx] = divisions + 1
else:
    
    def discretize(seq, divisions=divisions, range_=range_, seed=seed):
        '''
        >>> discretize([3, 4, 1, 8, 13, 8], divisions=4, range_=0, seed=123) = array([1, 1, 0, 2, 3, 2])
        where
        ranks = [2., 3., 1., 4.5, 6., 4.5]
        tresholds = [1.5,  3.,  4.5]
        '''
        np.random.seed(seed)
        ranks = rankdata(seq, method='average') # method='ordinal'/'average' ?

        random_blocks = np.cumsum(range_ * (2 * np.random.random(divisions + 1) - 1) + np.ones(divisions + 1))
        tresholds = random_blocks[:-1] / random_blocks[-1] * len(seq)

        discrete_seq = np.zeros(len(seq), dtype='uint8')
        for treshold in tresholds:
            discrete_seq[ranks > treshold] += 1
        return discrete_seq

    # discretize_vec = np.vectorize(discretize, signature='(n)->(n)', excluded=['divisions', 'range_', 'seed'])
    
    for col_idx in range(input_.shape[0] - 1):
        if col_idx in discrete_cols:
            data[col_idx] = input_[col_idx].astype('uint8')
            bucket_counts[col_idx] = len(np.unique(data[col_idx]))
        else:
            data[col_idx] = discretize(input_[col_idx])
            bucket_counts[col_idx] = divisions + 1


data[-1] = input_[-1:].astype('uint8')

final_results = {dof: {} for dof in set(map(np.prod, tuple(combinations(bucket_counts, r=k))))}

labels, counts = np.unique(data[-1], return_counts=True)
n_classes = len(labels)

xi = 0.25
pseudo_counts = xi * counts / np.min(counts)

dim0, dim1 = data[:-1].shape

# 3. More function definitions

def tuple_generator(k=k, dim0=dim0):
    '''
    Python-generator.
    E.g. output for k=2:
    {0,1}, {0,2}, ..., {0, dim0-1}, {1,2}, ..., {1,dim0-1}, ..., {dim0-2, dim0-1}
    Go with combinations_with_replacement() to include diagonal tuples like {0, 0}
    '''        
    return combinations(range(dim0), k)    

def neg_H(p):
    return p * np.log2(p)

def neg_H_cond(matrix):
    return np.sum(neg_H(matrix)) - np.sum(neg_H(np.sum(matrix, axis=-1)))

def slow_work(indeces, bucket_counts_):
    '''
    indeces -> tuple
    Work-function.
    Output: tuple of Information Gains implicitly corresponding to the indeces
    '''
    # contingency-matrix: begin with pseudo-counts
    contingency_m = np.empty(list(bucket_counts_) + [len(labels)], dtype='float64')
    for label, pseudo_count in enumerate(pseudo_counts):
        contingency_m[..., label] = pseudo_count
    
    # contingency-matrix: normal counts
    for c_index in data[list(indeces) + [-1]].T:
        contingency_m[tuple(c_index)] += 1
    
    results = []
    for i in range(len(indeces)):
        result = neg_H_cond(contingency_m) - neg_H_cond(np.sum(contingency_m, axis=i))
        results.append(result)
    return tuple(results)


def record(tuple_, IGs, records):
    for column, IG in zip(tuple_, IGs):
        if column not in records or IG > records[column][0]:
            records[column] = (IG, tuple_)


for tuple_ in tuple_generator():
    bucket_counts_ = tuple(bucket_counts[col_idx] for col_idx in tuple_)
    
    #IGs = slow_work(tuple_, bucket_counts_)
    IGs = fast.work_3a(dim1, bucket_counts_, data[tuple_[0]], data[tuple_[1]], data[tuple_[2]], n_classes, pseudo_counts, data[-1])

    dof = np.prod(bucket_counts_, dtype = 'int')
    record(tuple_, IGs, final_results[dof])

# result
print("Finished in", time() - time0, "sec.")

with open("march2_n1_results.pkl", "wb") as file:
    pickle.dump(final_results, file)

Overwriting march2_n1.py


In [12]:
%run march2_n1.py

Finished in 3.3211257457733154 sec.


In [13]:
import pandas as pd
import pickle
with open("march2_n1_results.pkl", "rb") as file:
    final_results_n1 = pickle.load(file)

for dof, dict_ in final_results_n1.items():
    
    df = pd.DataFrame(dict_).T.rename(columns={0: 'IG_max', 1: 'tuple'})
    df.sort_values('IG_max', ascending=False, inplace=True)

    print("product of X's bucket-counts = {}".format(dof))
    display(df)
    print("---")

product of X's bucket-counts = 128


,IG_max,tuple
1,273.836,"(1, 25, 43)"
22,268.547,"(22, 25, 43)"
43,221.95,"(1, 25, 43)"
25,221.267,"(1, 25, 43)"
3,205.555,"(3, 22, 25)"
14,124.049,"(6, 14, 25)"
6,121.529,"(6, 14, 25)"
42,118.579,"(0, 12, 42)"
12,118.251,"(0, 12, 42)"
21,117.97,"(21, 25, 32)"


---
product of X's bucket-counts = 512


,IG_max,tuple
43,469.58,"(1, 22, 43)"
3,457.692,"(1, 3, 22)"
22,412.102,"(3, 22, 41)"
1,407.952,"(1, 32, 43)"
9,392.656,"(7, 9, 14)"
18,390.184,"(18, 37, 47)"
7,383.371,"(7, 9, 14)"
14,379.428,"(7, 9, 14)"
38,373.26,"(6, 12, 38)"
48,372.887,"(12, 17, 48)"


---
product of X's bucket-counts = 64


...

---
product of X's bucket-counts = 192


,IG_max,tuple
43,284.9,"(1, 5, 43)"
22,278.071,"(5, 22, 43)"
3,265.697,"(1, 3, 5)"
1,257.315,"(1, 5, 43)"
21,166.651,"(5, 21, 27)"
5,166.641,"(5, 22, 43)"
26,166.587,"(5, 13, 26)"
28,165.338,"(17, 28, 30)"
17,163.5,"(17, 28, 30)"
41,162.253,"(22, 30, 41)"


---
product of X's bucket-counts = 72


,IG_max,tuple
43,103.065,"(5, 30, 43)"
22,101.528,"(5, 22, 30)"
25,95.2781,"(20, 25, 45)"
3,88.8426,"(3, 5, 30)"
5,67.4837,"(5, 22, 30)"
21,61.0716,"(5, 21, 30)"
9,60.6375,"(5, 9, 30)"
38,59.7251,"(5, 30, 38)"
41,56.4259,"(5, 30, 41)"
46,55.5272,"(5, 30, 46)"


---


In [6]:
%%bash
python -m cProfile -o march2_n1.prof march2_n1.py

Finished in 2.4379029273986816 sec.


In [7]:
import pstats
p = pstats.Stats('march2_n1.prof')
#p.sort_stats('time').print_stats(15)
p.strip_dirs().sort_stats('time').print_stats(10)

Sat Mar 23 13:56:07 2019    march2_n1.prof

         694558 function calls (684365 primitive calls) in 3.033 seconds

   Ordered by: internal time
   List reduced from 3386 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    19600    1.693    0.000    1.693    0.000 {built-in method fast.work_3a}
    40933    0.303    0.000    0.303    0.000 {method 'reduce' of 'numpy.ufunc' objects}
        1    0.202    0.202    3.035    3.035 march2_n1.py:2(<module>)
    39200    0.138    0.000    0.455    0.000 fromnumeric.py:2408(prod)
      539    0.065    0.000    0.065    0.000 {built-in method marshal.loads}
    822/1    0.048    0.000    3.035    3.035 {built-in method builtins.exec}
    19600    0.047    0.000    0.047    0.000 march2_n1.py:140(record)
   127/92    0.040    0.000    0.083    0.001 {built-in method _imp.create_dynamic}
    78400    0.031    0.000    0.031    0.000 march2_n1.py:147(<genexpr>)
1574/1559    0.029    0.000   

---
Make a script that uses mpi4py and run with mpi from bash

In [14]:
%%writefile march2_mpi.py
import csv
import numpy as np
from scipy.stats import rankdata
from itertools import product, chain, starmap, combinations, combinations_with_replacement
import pickle

import wrap_discretize
import fast

from mpi4py import MPI
comm = MPI.COMM_WORLD
comm.Barrier()
time0 = MPI.Wtime()
size = comm.Get_size()
rank = comm.Get_rank()

k = 3
window = 5
divisions = 7
range_ = 0.0
seed = 123
    
wrap_discretize_switch = True

# 1. Read in the data

file = "madelon_5_tiny.csv"
discrete_cols = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45]

input_ = []
with open(file) as csvfile:
    reader = csv.reader(csvfile, delimiter=',',
                        quoting=csv.QUOTE_NONNUMERIC)
    for row in reader:
        input_.append(row)
        
input_ = np.ascontiguousarray(np.array(input_, dtype='float64').T[:-1])
data = np.zeros_like(input_, dtype='uint8')
bucket_counts = np.empty(input_.shape[0] - 1, dtype='int')

# Discretize the data

if wrap_discretize_switch:

    for col_idx in range(input_.shape[0] - 1):
        if col_idx in discrete_cols:
            data[col_idx] = input_[col_idx].astype('uint8')
            bucket_counts[col_idx] = len(np.unique(data[col_idx]))
        else:
            wrap_discretize.discretize(
                seed = 123,
                discretization_index = 0,
                feature_id = col_idx,  # ?
                divisions = divisions,
                object_count = input_.shape[1],
                py_in_data = input_[col_idx],
                py_sorted_in_data = np.sort(input_[col_idx]),
                py_out_data = data[col_idx],
                range_ = range_
            )
            bucket_counts[col_idx] = divisions + 1
else:
    
    def discretize(seq, divisions=divisions, range_=range_, seed=seed):
        '''
        >>> discretize([3, 4, 1, 8, 13, 8], divisions=4, range_=0, seed=123) = array([1, 1, 0, 2, 3, 2])
        where
        ranks = [2., 3., 1., 4.5, 6., 4.5]
        tresholds = [1.5,  3.,  4.5]
        '''
        np.random.seed(seed)
        ranks = rankdata(seq, method='average') # method='ordinal'/'average' ?

        random_blocks = np.cumsum(range_ * (2 * np.random.random(divisions + 1) - 1) + np.ones(divisions + 1))
        tresholds = random_blocks[:-1] / random_blocks[-1] * len(seq)

        discrete_seq = np.zeros(len(seq), dtype='uint8')
        for treshold in tresholds:
            discrete_seq[ranks > treshold] += 1
        return discrete_seq

    #discretize_vec = np.vectorize(discretize, signature='(n)->(n)', excluded=['divisions', 'range_', 'seed'])
    
    for col_idx in range(input_.shape[0] - 1):
        if col_idx in discrete_cols:
            data[col_idx] = input_[col_idx].astype('uint8')
            bucket_counts[col_idx] = len(np.unique(data[col_idx]))
        else:
            data[col_idx] = discretize(input_[col_idx])
            bucket_counts[col_idx] = divisions + 1

    
data[-1] = input_[-1:].astype('uint8')

labels, counts = np.unique(data[-1], return_counts=True)
n_classes = len(labels)

xi = 0.25
pseudo_counts = xi * counts / np.min(counts)

dim0, dim1 = data[:-1].shape
M = (dim0 - 1) // window + 1
border_cols = range( (M-1) * window, dim0)

if rank == 0:
    print(rank, "dim0 =", dim0, "; dim1 =", dim1)

# 3. More function definitions

def tile_generator(k=k, M=M):
    '''
    Python-generator.
    E.g. output for k=2:
    {0,0}, {0,1}, ..., {0, M-1}, {1,1}, ..., {1,M-1}, ..., {M-1, M-1}
    Go with combinations(range(M), k) to exclude diagonal tuples
    '''        
    return combinations_with_replacement(range(M), k)    

def tuple_generator(tile, window=window, border_cols=border_cols):
    '''
    Map tile into sequence of k-tuples, i.e. fundamental-tiles,
    i.e. elements of the cartesian product of the data-columns
    '''
    index_counts = {index: tile.count(index) for index in tile}
    index_to_cols = lambda index: range(index * window, (index + 1) * window) if index != (M - 1) else border_cols 
    cols_tile = (combinations(index_to_cols(index), count) for (index, count) in index_counts.items())
    return (list(chain.from_iterable(col_indeces)) for col_indeces in product(*cols_tile))

def neg_H(p):
    return p * np.log2(p)

def neg_H_cond(matrix):
    return np.sum(neg_H(matrix)) - np.sum(neg_H(np.sum(matrix, axis=-1)))

def slow_work(tuple_, bucket_counts_):
    '''
    tuple_ -> list # dammit...
    Work-function.
    Output: tuple of Information Gains implicitly corresponding to column-indeces in the tuple_
    '''
    # contingency-matrix: begin with pseudo-counts
    contingency_m = np.empty(list(bucket_counts_) + [n_classes], dtype='float64')
    for label, pseudo_count in enumerate(pseudo_counts):
        contingency_m[..., label] = pseudo_count
    
    # contingency-matrix: normal counts
    for c_index in data[tuple_ + [-1]].T:
        contingency_m[tuple(c_index)] += 1
    
    IGs = tuple(neg_H_cond(contingency_m) - neg_H_cond(np.sum(contingency_m, axis=i)) for i in range(len(tuple_)))
    return IGs


def record_tuple(tuple_, IGs, dof, records):
    # records -> dict of dicts
    for column, IG in zip(tuple_, IGs):
        if dof not in records:
            records[dof] = {column: (IG, tuple_)}
        elif column not in records[dof] or IG > records[dof][column][0]:
            records[dof][column] = (IG, tuple_)
            
def record_tile(tile_results, records):
    # tile_results, records -> dicts of dicts
    for dof, dict_ in tile_results.items():
        for column, (IG, tuple_) in dict_.items():
            if column not in records[dof] or IG > records[dof][column][0]:
                records[dof][column] = (IG, tuple_)

# 4 Work loop

if rank == 0:
    final_results = {dof: {} for dof in set(map(np.prod, tuple(combinations(bucket_counts, r=k))))}
    current_assignements = {rank: (0, None) for rank in range(1,size)}
    
    print(rank, "entering the for loop")
    status = MPI.Status()
    for tile in tile_generator():
        tile_results = comm.recv(status=status)
        record_tile(tile_results, final_results)
        comm.isend(tile, dest=status.source)
        job_count = current_assignements[status.source][0]
        current_assignements[status.source] = (job_count + 1, tile)
        
        #print(rank, "currently:", current_assignements)
    
    print(rank, "Work queue is empty")
    for _ in range(size - 1):
        tile_results = comm.recv(status=status)
        record_tile(tile_results, final_results)
        comm.isend(None, dest=status.source)

    # Save the results to a file
    with open("march2_mpi_results.pkl", "wb") as file:
        pickle.dump(final_results, file)

    print(rank, "says goodbye")
    print(rank, "Elapsed:", MPI.Wtime() - time0, "sec")
    
else:
    comm.send({}, dest=0)
    print(rank, "entering the while loop")
    while True:
        tile = comm.recv(source = 0)
        try:
            tile_results = {}
            for tuple_ in tuple_generator(tile):
                bucket_counts_ = tuple(bucket_counts[col_idx] for col_idx in tuple_)
                
                #IGs = slow_work(tuple_, bucket_counts_)
                IGs = fast.work_3a(dim1, bucket_counts_, data[tuple_[0]], data[tuple_[1]], data[tuple_[2]], n_classes, pseudo_counts, data[-1])

                dof = np.prod(bucket_counts_, dtype = 'int')
                record_tuple(tuple_, IGs, dof, tile_results)
            comm.isend(tile_results, dest=0)
        except:
            print(rank, "says goodbye")
            break


Overwriting march2_mpi.py


In [15]:
%%bash
export OMP_NUM_THREADS=1
mpirun -n 2 python march2_mpi.py

1 entering the while loop
0 dim0 = 50 ; dim1 = 2000
0 entering the for loop
0 Work queue is empty
1 says goodbye
0 says goodbye
0 Elapsed: 2.1002594369929284 sec


In [16]:
import pandas as pd
import pickle
with open("march2_mpi_results.pkl", "rb") as file:
    final_results_mpi = pickle.load(file)

for dof, dict_ in final_results_mpi.items():
    
    df = pd.DataFrame(dict_).T.rename(columns={0: 'IG_max', 1: 'tuple'})
    df.sort_values('IG_max', ascending=False, inplace=True)

    print("product of X's bucket-counts = {}".format(dof))
    display(df)
    print("---")

product of X's bucket-counts = 128


,IG_max,tuple
1,273.836,"[1, 43, 25]"
22,268.547,"[43, 22, 25]"
43,221.95,"[1, 43, 25]"
25,221.267,"[1, 43, 25]"
3,205.555,"[3, 22, 25]"
14,124.049,"[6, 14, 25]"
6,121.529,"[6, 14, 25]"
42,118.579,"[0, 42, 12]"
12,118.251,"[0, 42, 12]"
21,117.97,"[21, 25, 32]"


---
product of X's bucket-counts = 512


,IG_max,tuple
43,469.58,"[1, 43, 22]"
3,457.692,"[1, 3, 22]"
22,412.102,"[3, 41, 22]"
1,407.952,"[1, 43, 32]"
9,392.656,"[7, 9, 14]"
18,390.184,"[47, 18, 37]"
7,383.371,"[7, 9, 14]"
14,379.428,"[7, 9, 14]"
38,373.26,"[6, 12, 38]"
48,372.887,"[48, 12, 17]"


---
product of X's bucket-counts = 64


...

---
product of X's bucket-counts = 192


,IG_max,tuple
43,284.9,"[1, 5, 43]"
22,278.071,"[43, 5, 22]"
3,265.697,"[1, 3, 5]"
1,257.315,"[1, 5, 43]"
21,166.651,"[5, 21, 27]"
5,166.641,"[43, 5, 22]"
26,166.587,"[5, 13, 26]"
28,165.338,"[17, 28, 30]"
17,163.5,"[17, 28, 30]"
41,162.253,"[41, 22, 30]"


---
product of X's bucket-counts = 72


,IG_max,tuple
43,103.065,"[43, 5, 30]"
22,101.528,"[5, 22, 30]"
25,95.2781,"[45, 20, 25]"
3,88.8426,"[3, 5, 30]"
5,67.4837,"[5, 22, 30]"
21,61.0716,"[5, 21, 30]"
9,60.6375,"[5, 9, 30]"
38,59.7251,"[5, 30, 38]"
41,56.4259,"[41, 5, 30]"
46,55.5272,"[5, 30, 46]"


---


Compare (manually) the results obtained with and without `mpi`

In [17]:
for key_mpi, dict_mpi in final_results_mpi.items():
    print(key_mpi)
    for column, result in dict_mpi.items():
        print("mpi: ", column, result)
        print(" n1: ", column, final_results_n1[key_mpi][column])
        print("--")
    print("--")

128
mpi:  0 (77.26722716582435, [0, 2, 22])
 n1:  0 (77.26722716582435, (0, 2, 22))
--
mpi:  1 (273.83602451232196, [1, 43, 25])
 n1:  1 (273.8360245123224, (1, 25, 43))
--
mpi:  2 (106.37081912634812, [2, 13, 25])
 n1:  2 (106.37081912634812, (2, 13, 25))
--
mpi:  3 (205.55459762078704, [3, 22, 25])
 n1:  3 (205.55459762078704, (3, 22, 25))
--
mpi:  4 (113.47304122762648, [4, 25, 33])
 n1:  4 (113.47304122762648, (4, 25, 33))
--
mpi:  6 (121.52879498169682, [6, 14, 25])
 n1:  6 (121.52879498169682, (6, 14, 25))
--
mpi:  7 (105.29950717678048, [4, 7, 25])
 n1:  7 (105.29950717678048, (4, 7, 25))
--
mpi:  8 (111.97358856251299, [0, 8, 21])
 n1:  8 (111.97358856251299, (0, 8, 21))
--
mpi:  9 (110.66435617403567, [0, 9, 39])
 n1:  9 (110.66435617403567, (0, 9, 39))
--
mpi:  10 (94.54683652560698, [10, 11, 35])
 n1:  10 (94.54683652560698, (10, 11, 35))
--
mpi:  11 (107.10007056154677, [10, 11, 35])
 n1:  11 (107.10007056154677, (10, 11, 35))
--
mpi:  12 (118.25052955909564, [0, 42, 12])
 

mpi:  26 (252.01506815265316, [43, 15, 26])
 n1:  26 (252.01506815265498, (15, 26, 43))
--
mpi:  27 (243.51400455032217, [40, 12, 27])
 n1:  27 (243.51400455032262, (12, 27, 40))
--
mpi:  28 (241.8223040290768, [42, 15, 28])
 n1:  28 (241.82230402907703, (15, 28, 42))
--
mpi:  29 (248.9346283113748, [14, 15, 29])
 n1:  29 (248.9346283113748, (14, 15, 29))
--
mpi:  31 (256.0365147396544, [40, 21, 31])
 n1:  31 (256.03651473965465, (21, 31, 40))
--
mpi:  32 (255.19867997728738, [44, 15, 32])
 n1:  32 (255.19867997728807, (15, 32, 44))
--
mpi:  33 (251.446989005123, [40, 7, 33])
 n1:  33 (251.44698900512185, (7, 33, 40))
--
mpi:  34 (228.64404694481686, [44, 15, 34])
 n1:  34 (228.64404694481755, (15, 34, 44))
--
mpi:  36 (231.05468543590382, [40, 31, 36])
 n1:  36 (231.0546854359029, (31, 36, 40))
--
mpi:  37 (230.80441798288734, [40, 11, 37])
 n1:  37 (230.80441798288894, (11, 37, 40))
--
mpi:  38 (244.21351910100088, [15, 22, 38])
 n1:  38 (244.21351910100088, (15, 22, 38))
--
mpi:  39

 n1:  33 (100.05167524418925, (5, 33, 45))
--
mpi:  34 (101.78747638613049, [5, 20, 34])
 n1:  34 (101.78747638613049, (5, 20, 34))
--
mpi:  35 (81.50727246696852, [45, 20, 35])
 n1:  35 (81.50727246697056, (20, 35, 45))
--
mpi:  36 (117.19444490268415, [45, 30, 36])
 n1:  36 (117.19444490268506, (30, 36, 45))
--
mpi:  37 (102.18010563797952, [5, 20, 37])
 n1:  37 (102.18010563797952, (5, 20, 37))
--
mpi:  38 (106.9443305813802, [5, 20, 38])
 n1:  38 (106.9443305813802, (5, 20, 38))
--
mpi:  39 (85.3624636525758, [20, 30, 39])
 n1:  39 (85.3624636525758, (20, 30, 39))
--
mpi:  41 (100.93556178503172, [41, 5, 20])
 n1:  41 (100.93556178502968, (5, 20, 41))
--
mpi:  42 (100.9199749796121, [42, 5, 45])
 n1:  42 (100.91997497961142, (5, 42, 45))
--
mpi:  43 (123.54526761274997, [43, 5, 20])
 n1:  43 (123.54526761275156, (5, 20, 43))
--
mpi:  44 (101.6727061374022, [44, 5, 45])
 n1:  44 (101.67270613740311, (5, 44, 45))
--
mpi:  45 (109.593933653759, [45, 30, 36])
 n1:  45 (109.593933653759

mpi:  13 (37.588019364549154, [5, 13, 30])
 n1:  13 (37.588019364549154, (5, 13, 30))
--
mpi:  14 (51.72876263123749, [5, 14, 30])
 n1:  14 (51.72876263123749, (5, 14, 30))
--
mpi:  16 (42.63649321552566, [5, 16, 30])
 n1:  16 (42.63649321552566, (5, 16, 30))
--
mpi:  17 (39.392862069913235, [5, 17, 30])
 n1:  17 (39.392862069913235, (5, 17, 30))
--
mpi:  18 (53.33438644823559, [5, 18, 30])
 n1:  18 (53.33438644823559, (5, 18, 30))
--
mpi:  19 (35.98750959614131, [5, 19, 30])
 n1:  19 (35.98750959614131, (5, 19, 30))
--
mpi:  20 (43.08922866091211, [5, 10, 20])
 n1:  20 (43.08922866091211, (5, 10, 20))
--
mpi:  21 (61.07155384317434, [5, 21, 30])
 n1:  21 (61.07155384317434, (5, 21, 30))
--
mpi:  22 (101.52813270819229, [5, 22, 30])
 n1:  22 (101.52813270819229, (5, 22, 30))
--
mpi:  23 (36.607376678676246, [5, 23, 30])
 n1:  23 (36.607376678676246, (5, 23, 30))
--
mpi:  24 (50.71220206809062, [5, 24, 30])
 n1:  24 (50.71220206809062, (5, 24, 30))
--
mpi:  25 (95.27807310229855, [45, 2